In [1]:
import numpy as np  # Library for numerical operations
import tifffile as tiff  # Library to read TIFF/hyperspectral files
from skimage import filters  # For image processing filters (Laplacian, Gaussian, etc.)
from ipywidgets import interact, IntSlider  # For interactive sliders in Jupyter Notebook
import matplotlib.pyplot as plt  # Library for plotting images

# Define wavelength range
num_bands = 204         # Total number of spectral bands (adjust according to dataset)
wavelength_start = 400  # Starting wavelength in nanometers
wavelength_end = 1000   # Ending wavelength in nanometers

# Create a list of wavelengths evenly spaced across the spectral range
wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)  # 400-1000 nm split into 204 bands

# Function to calculate metrics for a single band
def calculate_band_metrics(band_array):
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)  # Normalize to [0,1]
    
    brightness = np.nanmean(band_norm)  # Average intensity
    contrast = np.nanstd(band_norm)     # Standard deviation = contrast
    sharpness = filters.laplace(band_norm).var()  # Sharpness using Laplacian variance
    
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)  # Remove low-frequency content (Gaussian blur)
    noise = np.nanstd(highpass)  # Noise estimate from high-frequency components
    
    signal = np.sqrt(np.nanmean(band_norm**2))  # Signal power
    snr = 20 * np.log10(signal / (noise + 1e-8))  # Signal-to-Noise Ratio in dB
    
    return {
        "Brightness": brightness,
        "Contrast": contrast,
        "Sharpness": sharpness,
        "SNR(dB)": snr
    }

# Interactive function to display band metrics and image
def show_band_metrics_interactive(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"  # Construct image path
    try:
        cube = tiff.imread(file_path)  # Load hyperspectral cube (bands, rows, cols)
        bands, rows, cols = cube.shape  # Extract cube shape
        
        print(f"Loaded Image ID: {img_id}")  # Print image ID
        print(f"Image width (columns): {cols}")  # Print image width
        print(f"Image height (rows): {rows}")    # Print image height
        print(f"Number of bands (channels): {bands}")  # Print number of spectral bands
        print(f"Data type: {cube.dtype}")        # Print pixel data type
        bit_depth = cube.dtype.itemsize * 8      # Calculate bit depth
        print(f"Bit depth: {bit_depth}\n")       # Print bit depth
        
        cube = np.transpose(cube, (1, 2, 0))    # Rearrange cube to (rows, cols, bands) for visualization
        
        band_array = cube[:, :, band_idx]        # Extract the specific band array
        metrics = calculate_band_metrics(band_array)  # Calculate metrics for this band
        
        # Print wavelength info if available
        if len(wavelengths) == bands:
            band_wavelength = wavelengths[band_idx]  # Wavelength of the selected band
            print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength:.2f} nm")
        else:
            print(f"Selected Band: {band_idx} | Wavelength info not available")
        
        # Print calculated metrics
        print(f"Brightness: {metrics['Brightness']:.4f}")  # Brightness
        print(f"Contrast: {metrics['Contrast']:.4f}")      # Contrast
        print(f"Sharpness: {metrics['Sharpness']:.4f}")    # Sharpness
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")        # Signal-to-Noise Ratio
        
        # Display the band image
        plt.figure(figsize=(5,5))                        # Create figure
        plt.imshow(band_array, cmap='gray')              # Show band as grayscale image
        plt.title(f"Image ID: {img_id} | Band: {band_idx}")  # Add title
        plt.axis("off")                                  # Hide axes
        plt.show()                                       # Show the plot
        
    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")  # Handle missing files

# Dataset info and first image load
min_id = 0
max_id = 2409  # Total number of images in dataset (adjust if needed)

first_img_file = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff"  # Path for first image
first_cube = tiff.imread(first_img_file)               # Load first image
total_bands = first_cube.shape[0]                      # Determine total number of bands

# Create interactive sliders for Image ID and Band selection
interact(
    show_band_metrics_interactive,
    img_id=IntSlider(min=min_id, max=max_id, step=1, value=min_id, description='Image ID'),  # Image ID slider
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')       # Band index slider
)

interactive(children=(IntSlider(value=0, description='Image ID', max=2409), IntSlider(value=0, description='Ba…

<function __main__.show_band_metrics_interactive(img_id, band_idx)>